In [1]:
!pip install opencv-python-headless Pillow


In [8]:
from math import atan2, degrees

# Center of rotation
center = [194.5,191]

# Initial and final positions
initial_pos = [297,73]
final_pos = [293,72]

# Convert positions to vectors relative to the center of rotation
vec_initial = [initial_pos[0] - center[0], initial_pos[1] - center[1]]
vec_final = [final_pos[0] - center[0], final_pos[1] - center[1]]

# Calculate the angle of rotation using the atan2 of the cross product and dot product
angle_rad = atan2(vec_initial[0]*vec_final[1] - vec_initial[1]*vec_final[0], vec_initial[0]*vec_final[0] + vec_initial[1]*vec_final[1])
angle_deg = degrees(angle_rad)

# Since this is the total rotation over 10 frames, calculate per frame rotation
angle_per_frame = angle_deg / 751

angle_per_frame

-0.0018154528086769507

In [ ]:
from skimage import io
import numpy as np
import cv2

def de_rotate_tiff_skimage(input_path, output_path, rotation_center=(194.5,191), rotation_per_frame=angle_per_frame):
    """
    De-rotates each frame in a multi-frame TIFF file using skimage and saves the result.

    Parameters:
    - input_path: Path to the input multi-frame TIFF file.
    - output_path: Path where the de-rotated multi-frame TIFF file will be saved.
    - rotation_center: Tuple specifying the center of rotation (x, y).
    - rotation_per_frame: Rotation angle per frame in degrees (negative for clockwise correction).
    """
    # Read the multi-frame TIFF
    frames = io.imread(input_path)

    # Prepare an array to hold the de-rotated frames
    de_rotated_frames = np.zeros_like(frames)

    for frame_index in range(frames.shape[0]):
        # Extract the current frame
        frame = frames[frame_index]
        
        # Calculate the total rotation for the current frame
        total_rotation = rotation_per_frame * frame_index

        # Get the rotation matrix and apply the rotation
        rotation_matrix = cv2.getRotationMatrix2D(rotation_center, total_rotation, 1)
        de_rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))

        # Store the de-rotated frame
        de_rotated_frames[frame_index] = de_rotated_frame

    # Save the de-rotated frames as a multi-frame TIFF
    io.imsave(output_path, de_rotated_frames)
# Example usage
input_tiff_path = '/Volumes/T7red/silk_more/20240219/sample2/26min.tif'  # Update this path
output_tiff_path = '/Volumes/T7red/silk_more/20240219/sample2/derotated/26min_derotate.tif'  # Update this path

de_rotate_tiff_skimage(input_tiff_path, output_tiff_path)
